In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2 # https://ipython.org/ipython-doc/3/config/extensions/autoreload.html

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from keras.models import load_model


import sys
sys.path.insert(0, './api_tutorials')
from bqsr_cnn import parse_args, read_to_bqsr_tensor, bqsr_get_metric_dict, bqsr_bed_file_to_dict, BQSR_LABELS
from bqsr_cnn import bqsr_get_train_valid_test_paths_all, bqsr_label_tensors_generator, OQ_TENSOR_NAME, BQSR_TENSOR_NAME
from bqsr_cnn import label_bases_model_from_args
from recal_utils import tensor_to_quality_array, KL_divergence

Using TensorFlow backend.
/Users/tsato/anaconda3/envs/gatk/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
./api_tutorials/bqsr_cnn.py:28: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/tsato/anaconda3/envs/gatk/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/tsato/anaconda3/envs/gatk/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/tsato/anaconda3/envs/gatk/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instanc

In [2]:
sys.argv = ['bqsr_cnn']

args = parse_args()
args.skip_connections = [0, -1, -2, -3, -4, -5]
args.conv_layers = [128, 128, 128, 128, 128, 128]

Arguments are Namespace(activation='relu', activity_weight=1.0, annotation_set='_', annotations=[], bam_file=None, base_quality_mode='phot', batch_size=32, bed_file=None, channels_last=False, chrom=None, conv_batch_normalize=False, conv_dropout=0.0, conv_layers=[128, 96, 64, 48], conv_widths=[5, 5, 5, 5], data_dir=None, downsample_perfect_reads=0.0, end_pos=0, epochs=25, fc_batch_normalize=False, fc_dropout=0.0, fc_initializer='glorot_normal', fc_layers=[32], fps=1, id='no_id', ignore_vcf=None, input_symbol_set='dna_annotations', input_symbols={'A': 0, 'C': 1, 'G': 2, 'T': 3, 'pair': 4, 'cycle': 5, 'mq': 6}, inspect_model=False, inspect_show_labels=False, iterations=5, jitter=0.0, kernel_initializer='glorot_normal', l1=0.0, l2=0.0, label_smoothing=0.0, label_weights=[0.5, 0.5], labels={'GOOD_BASE': 0, 'BAD_BASE': 1}, learning_rate=0.01, map_input_to_logspace=False, max_pools=[], maxfun=9, mode=None, model=None, out='recalibrated.bam', output_dir='./weights/', padding='same', patience=4

In [3]:
model = label_bases_model_from_args(args)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bqsr (InputLayer)               (None, 151, 7)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 151, 128)     4608        bqsr[0][0]                       
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 151, 128)     82048       conv1d_1[0][0]                   
__________________________________________________________________________________________________
add_1 (Add)                     (None, 151, 128)     0           conv1d_2[0][0]                   
                                                                 conv1d_1[0][0]                   
__________

In [4]:
i=1
args.skip_connections[3]

-3